In [30]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role, image_uris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer
from sagemaker.session import s3_input, Session
#import quandl

In [32]:
my_region = boto3.session.Session().region_name
print(my_region)
bucket_name = 'sagemakerml'

us-east-1


In [19]:
#bucket_name = 'sagemakerml' # <--- NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [3]:
dataset=pd.read_csv('movie_rate.csv')

In [5]:
dataset.head()

,Movie,Year,Rating,Genre,Gross,Budget,Screens,Sequel,Sentiment,Views,Likes,Dislikes,Comments,Follow
0,13 Sins,2014,6.3,8,9130,4000000.0,45.0,1,0,3280543,4632,425,636,1120000.0
1,22 Jump Street,2014,7.1,1,192000000,50000000.0,3306.0,2,2,583289,3465,61,186,12350000.0
2,3 Days to Kill,2014,6.2,1,30700000,28000000.0,2872.0,1,0,304861,328,34,47,483000.0
3,300: Rise of an Empire,2014,6.3,1,106000000,110000000.0,3470.0,2,0,452917,2429,132,590,568000.0
4,A Haunted House 2,2014,4.7,8,17300000,3500000.0,2310.0,2,0,3145573,12163,610,1082,1923800.0


In [6]:
dataset.describe()

,Year,Rating,Genre,Gross,Budget,Screens,Sequel,Sentiment,Views,Likes,Dislikes,Comments,Follow
count,220.000000,220.000000,220.000000,2.200000e+02,2.190000e+02,211.000000,220.000000,220.000000,2.200000e+02,220.000000,220.000000,220.000000,1.960000e+02
mean,2014.259091,6.435455,5.340909,6.966363e+07,4.840684e+07,2207.118483,1.372727,2.568182,3.694327e+06,12811.190909,673.868182,1831.986364,3.038193e+06
std,0.439135,0.997402,4.143230,9.001975e+07,5.522917e+07,1470.525899,0.987216,6.915002,4.566679e+06,29480.797832,1248.741279,3643.409405,4.886278e+06
min,2014.000000,3.100000,1.000000,2.470000e+03,7.000000e+04,2.000000,1.000000,-38.000000,6.980000e+02,1.000000,0.000000,0.000000,1.066000e+03
25%,2014.000000,5.800000,1.000000,9.822500e+06,8.966390e+06,440.500000,1.000000,0.000000,6.049710e+05,1606.750000,105.750000,247.250000,1.830250e+05
50%,2014.000000,6.500000,3.000000,4.060000e+07,2.700000e+07,2776.000000,1.000000,0.000000,2.391182e+06,5868.500000,340.500000,817.000000,1.052600e+06
75%,2015.000000,7.100000,8.000000,9.057500e+07,6.500000e+07,3374.000000,1.000000,5.000000,5.137576e+06,14827.500000,694.250000,2100.250000,3.694500e+06
max,2015.000000,8.700000,15.000000,6.430000e+08,2.500000e+08,4324.000000,7.000000,29.000000,3.262678e+07,370552.000000,13960.000000,38363.000000,3.103000e+07


In [7]:
dataset.shape

(220, 14)

In [18]:
dataset.iloc[:200,2:].to_csv('train.csv',index=False,header=False)
dataset.iloc[201:,2:].to_csv('test.csv',index=False,header=False)

In [22]:
dd=pd.read_csv('train.csv')
dd.head()

,6.3,8,9130,4000000.0,45.0,1,0,3280543,4632,425,636,1120000.0
0,7.1,1,192000000,50000000.0,3306.0,2,2,583289,3465,61,186,12350000.0
1,6.2,1,30700000,28000000.0,2872.0,1,0,304861,328,34,47,483000.0
2,6.3,1,106000000,110000000.0,3470.0,2,0,452917,2429,132,590,568000.0
3,4.7,8,17300000,3500000.0,2310.0,2,0,3145573,12163,610,1082,1923800.0
4,4.6,3,29000,500000.0,NaN,1,0,91137,112,7,1,310000.0


In [26]:
# set an output path where the trained model will be saved
prefix = 'ml^2'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://sagemakerml/ml^2/output


In [37]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [35]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [42]:
# create pretrained container
container = image_uris.retrieve(region=boto3.Session().region_name, framework="linear-learner")

In [48]:
#Hyper perameter fit

sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(
    container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.c4.xlarge",
    output_path=output_path,
    sagemaker_session=sess,
)
linear.set_hyperparameters(feature_dim=784, predictor_type="regressor", mini_batch_size=200)

In [49]:
linear.fit({'train': s3_input_train,'validation': s3_input_test})

2022-06-01 14:41:41 Starting - Starting the training job...
2022-06-01 14:42:08 Starting - Preparing the instances for trainingProfilerReport-1654094501: InProgress
.........
2022-06-01 14:43:35 Downloading - Downloading input data...
2022-06-01 14:43:55 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[06/01/2022 14:44:29 INFO 140501399557952] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'auto', 'margin': '1.0', 'quantile': '0.5', 'loss_ins

UnexpectedStatusException: Error for Training job linear-learner-2022-06-01-14-41-41-755: Failed. Reason: ClientError: No iterator has been registered for ContentType ('csv', '1.0'), exit code: 2